In [62]:
import sys
from sys import stderr

import numpy as np
import six
from keras.callbacks import Callback
from tqdm import tqdm, tqdm_notebook

class TQDMCallback(Callback):
    def __init__(self, outer_description="Training",
                 inner_description_initial="Epoch: {epoch}",
                 inner_description_update="Epoch: {epoch} - {metrics}",
                 metric_format="{name}: {value:0.3f}",
                 separator=", ",
                 leave_inner=True,
                 leave_outer=True,
                 show_inner=True,
                 show_outer=True,
                 output_file=stderr,
                 initial=0):

        self.outer_description = outer_description
        self.inner_description_initial = inner_description_initial
        self.inner_description_update = inner_description_update
        self.metric_format = metric_format
        self.separator = separator
        self.leave_inner = leave_inner
        self.leave_outer = leave_outer
        self.show_inner = show_inner
        self.show_outer = show_outer
        self.output_file = output_file
        self.tqdm_outer = None
        self.tqdm_inner = None
        self.epoch = None
        self.running_logs = None
        self.inner_count = None
        self.initial = initial

    def tqdm(self, desc, total, leave, initial=0):
        return tqdm(desc=desc, total=total, leave=leave, file=self.output_file, initial=initial)

    def build_tqdm_outer(self, desc, total):
        return self.tqdm(desc=desc, total=total, leave=self.leave_outer, initial=self.initial)

    def build_tqdm_inner(self, desc, total):
        return self.tqdm(desc=desc, total=total, leave=self.leave_inner)

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch = epoch
        desc = self.inner_description_initial.format(epoch=self.epoch)
        self.mode = 0  # samples
        if 'samples' in self.params:
            self.inner_total = self.params['samples']
        elif 'nb_sample' in self.params:
            self.inner_total = self.params['nb_sample']
        else:
            self.mode = 1  # steps
            self.inner_total = self.params['steps']
        if self.show_inner:
            self.tqdm_inner = self.build_tqdm_inner(desc=desc, total=self.inner_total)
        self.inner_count = 0
        self.running_logs = {}

    def on_epoch_end(self, epoch, logs={}):
        metrics = self.format_metrics(logs)
        desc = self.inner_description_update.format(epoch=epoch, metrics=metrics)
        if self.show_inner:
            self.tqdm_inner.desc = desc
            # set miniters and mininterval to 0 so last update displays
            self.tqdm_inner.miniters = 0
            self.tqdm_inner.mininterval = 0
            self.tqdm_inner.update(self.inner_total - self.tqdm_inner.n)
            self.tqdm_inner.close()
        if self.show_outer:
            self.tqdm_outer.update(1)        

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_batch_end(self, batch, logs={}):
        if self.mode == 0:
            update = logs['size']
        else:
            update = 1
        self.inner_count += update
        if self.inner_count < self.inner_total:
            self.append_logs(logs)
            metrics = self.format_metrics(self.running_logs)
            desc = self.inner_description_update.format(epoch=self.epoch, metrics=metrics)
            if self.show_inner:
                self.tqdm_inner.desc = desc
                self.tqdm_inner.update(update)

    def on_train_begin(self, logs={}):
        if self.show_outer:
            epochs = (self.params['epochs'] if 'epochs' in self.params
                      else self.params['nb_epoch'])
            self.tqdm_outer = self.build_tqdm_outer(desc=self.outer_description,
                                                    total=epochs)

    def on_train_end(self, logs={}):
        if self.show_outer:
            self.tqdm_outer.close()

    def append_logs(self, logs):
        metrics = self.params['metrics']
        for metric, value in six.iteritems(logs):
            if metric in metrics:
                if metric in self.running_logs:
                    self.running_logs[metric].append(value[()])
                else:
                    self.running_logs[metric] = [value[()]]

    def format_metrics(self, logs):
        metrics = self.params['metrics']
        strings = [self.metric_format.format(name=metric, value=np.mean(logs[metric], axis=None)) for metric in metrics
                   if
                   metric in logs]
        return self.separator.join(strings)
    
    

class TQDMNotebookCallback(TQDMCallback):
    def __init__(self,
                 outer_description="Training",
                 inner_description_initial="Epoch {epoch}",
                 inner_description_update="[{metrics}] ",
                 metric_format="{name}: {value:0.3f}",
                 separator=", ",
                 leave_inner=False,
                 leave_outer=True,
                 output_file=sys.stderr,
                 initial=0, **kwargs):
        super(TQDMNotebookCallback, self).__init__(outer_description=outer_description,
                                                   inner_description_initial=inner_description_initial,
                                                   inner_description_update=inner_description_update,
                                                   metric_format=metric_format,
                                                   separator=separator,
                                                   leave_inner=leave_inner,
                                                   leave_outer=leave_outer,
                                                   output_file=output_file,
                                                   initial=initial, **kwargs)

    def tqdm(self, desc, total, leave, initial=0):
        return tqdm_notebook(desc=desc, total=total, leave=leave, initial=initial)
    

In [63]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth=True

sess = tf.Session(config=config)

import numpy as np
from PIL import Image
from keras import callbacks
# from keras.layers import Conv2D, BatchNormalization, Input, Activation, UpSampling2D, Lambda, Deconv2D
from keras.layers import Conv2D, BatchNormalization, Input, Activation, UpSampling2D, Lambda
from keras.layers.convolutional import Deconv2D
from keras.layers.merge import add
from keras.models import Model
from keras import backend as K
# from keras_tqdm import TQDMNotebookCallback
# from tqdm import TQDMNotebookCallback
from vgg16_avg import VGG16_Avg
from matplotlib import pyplot as plt
from keras.models import load_model
from sklearn.model_selection import train_test_split

%matplotlib inline

In [64]:
# GPU check

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10390601803155334164, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6668420649
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10101611027989538602
 physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [65]:
# Need to install a Javascript widget for this to work
params = {'verbose': 0, 'callbacks': [TQDMNotebookCallback(leave_inner=True)]}
lr_img_dir = 'data/lr_images/'
hr_img_dir = 'data/hr_images/'
fnames = os.listdir('data/mirflickr/')
lr_dims = (72, 72, 3) # Number of channels is last dim with tf backend
hr_dims = (288, 288, 3)

# lr_dims = (128, 128, 3)
# hr_dims = (400, 400, 3) # 4X times better resolution


The CNN architechture used will be similar to ResNets.
- Has convolutional blocks.
- Has residual connections and hence need residual blocks.
- Upsampling would be done using de-convolution blocks.
(Instead of deconvolution block, we could use UpSampling2D from keras)

In [66]:
def conv_block(x, filters, size, stride=(2,2), mode='same', act=True):
    x = Conv2D(filters, kernel_size=(size, size), strides=stride, padding=mode)(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x) if act else x

def res_block(ip, nf=64):
    x = conv_block(ip, nf, 3, (1,1))
    x = conv_block(x, nf, 3, (1,1), act=False)
    return add([x, ip]) # Adding the transformed feature map with the original input (residual connection)

def deconv_block(x, filters, size, stride=(2,2)):
    x = Deconv2D(filters, kernel_size=(size, size), strides=stride, 
        padding='same')(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x)

def up_block(x, filters, size):
    x = UpSampling2D()(x)
    x = Conv2D(filters, kernel_size=(size, size), padding='same')(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x)

inp = Input(shape=lr_dims)
x = conv_block(inp, 64, 9, (1,1))
for i in range(4): 
    x = res_block(x)
x = up_block(x, 64, 3) # Try up-sampling vs. deconv instead (deconv might produce checkerboard patterns)
x = up_block(x, 64, 3)
x = Conv2D(3, kernel_size=(9, 9), activation='tanh', padding='same')(x)
outp = Lambda(lambda x: (x+1)*127.5)(x) # +1 since tanh gives values between -1 and 1, and we need to scale to 0

Basically, our model takes a Low resolution 100x100 image, which is transformed through a series of conv blocks and residual connections. Then we start to up-sample/deconv the feature map to make it of size 400x400. 

Finally, we compare the 'content loss' between the generated image and the ground truth High resolution 400x400 image, when both are passed through some convolutional layer in Vgg16. In doing so, we are able to train a network that can upsample an image and recreate the higher resolution details.

In [67]:
base_model = Model(inputs=inp, outputs=outp)
base_model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 72, 72, 3)    0                                            
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 72, 72, 64)   15616       input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_34 (BatchNo (None, 72, 72, 64)   256         conv2d_37[0][0]                  
__________________________________________________________________________________________________
activation_22 (Activation)      (None, 72, 72, 64)   0           batch_normalization_34[0][0]     
___________________________________________________________________________________________

Total params: 403,267
Trainable params: 401,859
Non-trainable params: 1,408
__________________________________________________________________________________________________


In [68]:
# Pre-processing and de-preprocessing as done in the style transfer tutorial
rn_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)

preproc = lambda x: (x - rn_mean)[:, :, :, ::-1]
deproc = lambda x,s: np.clip(x.reshape(s)[:, :, :, ::-1] + rn_mean, 0, 255)

In [69]:
im = Image.open('data/lr_images/im002.jpg')
img_arr = preproc(np.expand_dims(im, axis=0))
shp = img_arr.shape
shp

(1, 72, 72, 3)

In [70]:
vgg_inp = Input(shape=hr_dims)
vgg = VGG16_Avg(include_top=False, input_tensor=Lambda(preproc)(vgg_inp))

vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 288, 288, 3)       0         
_________________________________________________________________
lambda_23 (Lambda)           (None, 288, 288, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 288, 288, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 288, 288, 64)      36928     
_________________________________________________________________
block1_pool (AveragePooling2 (None, 144, 144, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 144, 144, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 144, 144, 128)     147584

Since we only want to learn the "upsampling network" (base_model), and are just using VGG to calculate the loss function, we set the VGG layers to not be trainable.

In [71]:
for l in vgg.layers: 
    l.trainable=False

In [72]:
# A function to return a model's output at a specified layer number (basically for VGG)
def get_outp(model, ln): 
    return model.get_layer(f'block{ln}_conv1').output

vgg_content = Model(vgg_inp, [get_outp(vgg, o) for o in [1,2,3]]) # Layers 1,2,3

# Two VGG networks
# 1. One that takes the original ground truth HRes image as input
# 2. Second that takes the generated HRes image as input
vgg1 = vgg_content(Lambda(preproc)(vgg_inp))
vgg2 = vgg_content(Lambda(preproc)(outp))

In [73]:
# Calculate the MSE, and the ouput has shape (1, batch_size)
def mean_sqr_b(diff): 
    dims = list(range(1,K.ndim(diff)))
    return K.expand_dims(K.sqrt(K.mean(diff**2, axis=dims)), axis=0)

# Test the above with numpy code
t = np.ones(shape=(8,200,200,3))
dims = list(range(1, np.ndim(t)))
K.eval(K.expand_dims(K.sqrt(K.mean(K.variable(t)**2, axis=dims)), axis=0)).shape

(1, 8)

In [74]:
w=[0.1, 0.8, 0.1] # Considering conv blocks 1,2,3 - different weights for each
def content_loss_fn(x): # x is an array of VGG1 outputs appended with an array of VGG2 outputs
    res = 0; n=len(w)
    for i in range(n): # Iterate through the 3 conv blocks
        # Below, we compare x[0] and x[3] | x[1] and x[4] and so on
        # x[0], x[1], x[2] are outputs of vgg1
        # x[3], x[4], x[5] are outputs of vgg2
        res += mean_sqr_b(x[i]-x[i+n]) * w[i]
    return res

We also define a zero vector as a target parameter, which is a **necessary parameter when calling fit on a keras model.**

In [75]:
model_sr = Model([inp, vgg_inp], Lambda(content_loss_fn)(vgg1+vgg2))# array of VGG1 outputs appended with an array of VGG2 outputs
targ = np.zeros(shape=list( hr_dims ) + [1]) # From FastAI notebook
# targ = np.zeros(shape=(BATCH_SIZE,)) # This is used in my batch_generator function
# Target and input 1st dimensions should match

model_sr.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 72, 72, 3)    0                                            
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 72, 72, 64)   15616       input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_34 (BatchNo (None, 72, 72, 64)   256         conv2d_37[0][0]                  
__________________________________________________________________________________________________
activation_22 (Activation)      (None, 72, 72, 64)   0           batch_normalization_34[0][0]     
___________________________________________________________________________________________

__________________________________________________________________________________________________
lambda_22 (Lambda)              (None, 288, 288, 3)  0           conv2d_48[0][0]                  
__________________________________________________________________________________________________
lambda_24 (Lambda)              (None, 288, 288, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
lambda_25 (Lambda)              (None, 288, 288, 3)  0           lambda_22[0][0]                  
__________________________________________________________________________________________________
model_11 (Model)                [(None, 288, 288, 64 555328      lambda_24[0][0]                  
                                                                 lambda_25[0][0]                  
__________________________________________________________________________________________________
lambda_26 

#### Fit Generator and load data in batches

In [77]:
def load_batch(file_list):
    lr_img_array = []
    hr_img_array = []
    
    for file_ in file_list:
        lr_im = Image.open(lr_img_dir + file_)
        lr_img_array.append(np.array(lr_im))
        
        hr_im = Image.open(hr_img_dir + file_)
        hr_img_array.append(np.array(hr_im))
        
    return lr_img_array, hr_img_array

def batch_generator(files, BATCH_SIZE):
    L = len(files)

    #this line is just to make the generator infinite, keras needs that    
    while True:

        batch_start = 0
        batch_end = BATCH_SIZE

        while batch_start < L:
            
            limit = min(batch_end, L)
            file_list = files[batch_start: limit]
            lr_img_array, hr_img_array = load_batch(file_list)
            targ = np.zeros(shape=(BATCH_SIZE,))
            
            yield [np.array(lr_img_array), np.array(hr_img_array)], targ # a tuple with two numpy arrays with batch_size samples     

            batch_start += BATCH_SIZE   
            batch_end += BATCH_SIZE


fnames = [f for f in fnames if f.endswith('jpg')]
print(len(fnames))

train_files, _temp = train_test_split(fnames, test_size = 400)
dev_files, test_files = train_test_split(_temp, test_size = 0.5)

100006


In [82]:
# parameters

BATCH_SIZE = 10
N_EPOCHS = 2
STEPS_PER_EPOCH = len(train_files)//BATCH_SIZE
VAL_STEPS = len(dev_files)//BATCH_SIZE

# The final layer returns losses as shape (1, batch_size) 
# - the loss is set to 'mae' to take mean average error across the batch
model_sr.compile(optimizer='adam', loss='mse') 
# model_sr.fit(x=[arr_lr, arr_hr], y=targ, batch_size=8, epochs=3)

history = model_sr.fit_generator(
    generator = batch_generator(train_files, BATCH_SIZE),
    epochs = N_EPOCHS,
    steps_per_epoch = STEPS_PER_EPOCH,
    validation_data = batch_generator(dev_files, BATCH_SIZE), 
    validation_steps = VAL_STEPS,
)

In [79]:
base_model.save_weights('saved_models/sr_upsampling_epoch_2_weights.h5')